# Real-Time Prediction

## Usage

With this notebook, real-time predictions can be made on signs you perform in front of your running webcam. 

### <span style="color:green">Quick User Guide: 

<span style="color:green">1. Load your trained TF Model.</span>

<span style="color:green">2. Copy + paste used Configurations from the pre-processing notebook</span>

<span style="color:green">Optional: 3. Copy + paste used Pre-processing Layer from the pre-processing notebook</span>


### Game Menu
![image](../images/Real-Time-Demo-Menu.png)

## Install and Import Dependencies

### Install Dependencies

In [676]:
%pip install tensorflow-macos opencv-python mediapipe-silicon sklearn matplotlib
#!pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python mediapipe sklearn matplotlib # original code line from tutorial (he had a windows system)

Note: you may need to restart the kernel to use updated packages.


### Import Dependencies

In [677]:
# general
import numpy as np
import pandas as pd
import os # easier file path handling

# for camera feed
import cv2 # opencv
from matplotlib import pyplot as plt # imshow for easy visualization
import time # to insert "sleep" in between frames
import mediapipe as mp # for accessing and reading from webcam

# for model re-building
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking

# for loading .json dictionary
import json

## <span style="color:green">Load Files</span>

### <span style="color:green">Load Saved Model</span>

#### <span style="color:green">Here, you can load your trained TensorFlow model, e.g. LSTM.</span>

In [678]:
# load model
model = tf.keras.models.load_model('../models/LSTM/LSTM_model_20signsz_8.h5')

### Load sign_to_prediction_index_map.json file

In [679]:
# label map dictionary 
#LABEL_MAP = json.load(open("../data/asl-signs/sign_to_prediction_index_map.json", "r")) 
LABEL_MAP = {'brown': 0,  'call(phone)': 1,  'cow': 2,  'cry': 3,  'dad': 4,  'fireman': 5,  'frog': 6,  'gum': 7,  'ice cream': 8,  'my/mine': 9,  'nose': 10,  'owl': 11,  'please': 12,  'radio': 13,  'quiet': 14,  'shirt': 15,  'tomorrow': 16,  'uncle': 17,  'water': 18,  'who': 19}

## Setup

### Game Mechanics

In [680]:
COUNTDOWN = 0
#SELECTED_SIGNS = ['hello', 'mom', 'dad', 'hungry', 'thirsty']
#SELECTED_SIGNS = ['apple', 'bath', 'owl', 'clown', 'better']
SELECTED_SIGNS = list(LABEL_MAP.keys()) 

LEVEL = [['frog', 'cow', 'quiet', 'nose', 'dad'], ['brown', 'fireman', 'call(phone)', 'please', 'my/mine']]
#LEVEL = [['frog', 'cow', 'quiet', 'nose', 'dad']] 

### <span style="color:green">Preprocessing Setup </span>

#### <span style="color:green">These configuration must be the same ones, which you used to preprocess the data to train the TensorFlow model, which you loaded above! </span>

In [681]:
#Define length of sequences for padding or cutting; 22 is the median length of all sequences
LENGTH = 22

#final data will be flattened, if false data will be 3 dimensional
FLATTEN = False

#Define padding mode 
#1 = padding at start&end; 2 = padding at end)
PADDING = 2
CONSTANT_VALUE = 0 

#define if z coordinate will be dropped
DROP_Z = False
DEFINE_HAND = True

#Defining Landmarks
#index ranges for each landmark type
#dont change these landmarks
FACE = list(range(0, 468))
LEFT_HAND = list(range(468, 489))
POSE = list(range(489, 522))
POSE_UPPER = list(range(489, 510))
RIGHT_HAND = list(range(522, 543))
LIPS = [61, 185, 40, 39, 37,  0, 267, 269, 270, 409,
                 291,146, 91,181, 84, 17, 314, 405, 321, 375, 
                 78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 
                 95, 88, 178, 87, 14,317, 402, 318, 324, 308]
lipsUpperOuter= [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter= [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner= [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner= [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
#defining landmarks that will be merged
averaging_sets = []

#generating list with all landmarks selected for preprocessing
#change landmarks you want to use here:
point_landmarks_right = RIGHT_HAND + lipsUpperInner + lipsLowerInner
point_landmarks_left = LEFT_HAND + lipsUpperInner + lipsLowerInner

#calculating sum of total landmarks used
LANDMARKS = len(point_landmarks_right) + len(averaging_sets)
print(f'Total count of used landmarks: {LANDMARKS}')

#defining input shape for model
if DROP_Z:
    INPUT_SHAPE = (LENGTH,LANDMARKS*2)
else:
    INPUT_SHAPE = (LENGTH,LANDMARKS*3)
print(INPUT_SHAPE)

Total count of used landmarks: 43
(22, 129)


### Visualization

In [682]:
# colors used
color_turquoise = (177, 195, 0)
color_coral = (81, 99, 241)
color_bg = (242, 248, 255)

### Objects and Functions for MP Holistic Keypoints

#### Initialize MP Holistic Model

In [683]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

#### Define Functions

##### mediapipe_detection()

In [684]:
# function to detect MP Holistic landmarks from an image, e.g. frames of your camera feed
def mediapipe_detection(image, model): 
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # color conversion BGR to RGB
    image.flags.writeable = False                   # image no longer writeable
    results = model.process(image)                  # make prediction
    image.flags.writeable = True                    # image is writeable again
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # color conversion back to original
    return image, results

##### draw_styled_landmarks()

In [685]:
# function to draw landmarks points and connecting lines on top of an image, e.g. on top of your camera feed
def draw_styled_landmarks(image, results): 
    # draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                              mp_drawing.DrawingSpec(color=color_turquoise, thickness=1, circle_radius=1), 
                              mp_drawing.DrawingSpec(color=color_turquoise, thickness=1, circle_radius=1))
    # draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=color_bg, thickness=2, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=color_bg, thickness=2, circle_radius=2)) 
    # draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=color_coral, thickness=2, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=color_coral, thickness=2, circle_radius=2)) 
    # draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                              mp_drawing.DrawingSpec(color=color_coral, thickness=2, circle_radius=4), 
                              mp_drawing.DrawingSpec(color=color_coral, thickness=2, circle_radius=2))

##### extract_keypoints()

In [686]:
# function to extract coordinates (+visibility) of all landmarks --> keypoints
# and concatenates everything into a flattened list 
def extract_keypoints(results): 
    face = np.array([[r.x, r.y, r.z] for r in results.face_landmarks.landmark]) if results.face_landmarks else np.zeros([468, 3])
    left_hand = np.array([[r.x, r.y, r.z] for r in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros([21, 3])
    pose = np.array([[r.x, r.y, r.z] for r in results.pose_landmarks.landmark]) if results.pose_landmarks else np.zeros([33, 3]) # x, y, z and extra value visibility
    right_hand = np.array([[r.x, r.y, r.z] for r in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.zeros([21, 3])
    return np.concatenate([face, left_hand, pose, right_hand]) # original code
    # a flattened list with list of all pose, face, left_hand, right_hand landmark x, y, z, (+visibility) coordinates

##### prob_viz()

In [687]:
# function to visualize predicted word probabilities with a dynamic real-time bar chart
def prob_viz(pred, image, SELECTED_SIGNS): 
    output_frame = image.copy() 
    bar_left = 10
    bar_text_space = 5
    text_left = bar_left + bar_text_space
    bar_top = 140
    bar_height = 20
    bar_bottom = bar_top + bar_height
    alpha = 0.3  # Transparency factor

    overlay = output_frame.copy()
    cv2.rectangle(output_frame, (0, 120), (230, 720), color_bg, -1)
    # Following line overlays transparent rectangle over the image
    output_frame = cv2.addWeighted(overlay, alpha, output_frame, 1 - alpha, 0)
    
    for num, prob in enumerate(pred): 
        cv2.rectangle(output_frame, 
                      pt1=(bar_left, bar_top+num*27), 
                      pt2=(bar_left+int(prob*100), bar_bottom+num*27), 
                      color=color_turquoise, thickness=-1)

        cv2.putText(img=output_frame, 
                    text=SELECTED_SIGNS[num], 
                    org=(text_left, bar_bottom-3+num*27), 
                    fontFace=cv2.FONT_ITALIC, fontScale=0.75, 
                    color=(50, 50, 50), 
                    thickness=1, lineType=cv2.LINE_AA)
    return output_frame

##### Helper Function - Preprocessing

In [688]:
def tf_nan_mean(x, axis=0):
    #calculates the mean of a TensorFlow tensor x along a specified axis while ignoring any NaN values in the tensor.
    return tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis) / tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis)

def right_hand_percentage(x):
    #calculates percentage of right hand usage
    right = tf.gather(x, RIGHT_HAND, axis=1)
    left = tf.gather(x, LEFT_HAND, axis=1)
    right_count = tf.reduce_sum(tf.where(tf.math.is_nan(right), tf.zeros_like(right), tf.ones_like(right)))
    left_count = tf.reduce_sum(tf.where(tf.math.is_nan(left), tf.zeros_like(left), tf.ones_like(left)))
    return right_count / (left_count+right_count)

In [689]:
#generating preprocessing layer that will be added to final model
class FeatureGen(tf.keras.layers.Layer):
    #defines custom tensorflow layer 
    def __init__(self):
        #initializes layer
        super(FeatureGen, self).__init__()
    
    def call(self, x_in, MIRROR=False):
        #drop z coordinates if required
        if DROP_Z:
            x_in = x_in[:, :, 0:2]
        if MIRROR:
            #flipping x coordinates
            x_in = np.array(x_in)
            x_in[:, :, 0] = (x_in[:, :, 0]-1)*(-1)
            x_in = tf.convert_to_tensor(x_in)

        #generates list with mean values for landmarks that will be merged
        x_list = [tf.expand_dims(tf_nan_mean(x_in[:, av_set[0]:av_set[0]+av_set[1], :], axis=1), axis=1) for av_set in averaging_sets]
        
        #extracts specific columns from input x_in defined by landmarks
        if DEFINE_HAND:
            handedness = right_hand_percentage(x_in)
            if handedness > 0.5:
                x_list.append(tf.gather(x_in, point_landmarks_right, axis=1))
            else: 
                x_list.append(tf.gather(x_in, point_landmarks_left, axis=1))
        else:
            x_list.append(tf.gather(x_in, point_landmarks, axis=1))
        #concatenates the two tensors from above along axis 1/columns
        x = tf.concat(x_list, 1)

        #padding to desired length of sequence (defined by LENGTH)
        #get current number of rows
        x_padded = x
        current_rows = tf.shape(x_padded)[0]
        #if current number of rows is greater than desired number of rows, truncate excess rows
        if current_rows > LENGTH:
            x_padded = x_padded[:LENGTH, :, :]

        #if current number of rows is less than desired number of rows, add padding
        elif current_rows < LENGTH:
            #calculate amount of padding needed
            pad_rows = LENGTH - current_rows

            if PADDING ==4: #copy first/last frame
                if pad_rows %2 == 0: #if pad_rows is even
                    padding_front = tf.repeat(x_padded[0:1, :], pad_rows//2, axis=0)
                    padding_back = tf.repeat(x_padded[-1:, :], pad_rows//2, axis=0)
                else: #if pad_rows is odd
                    padding_front = tf.repeat(x_padded[0:1, :], (pad_rows//2)+1, axis=0)
                    padding_back = tf.repeat(x_padded[-1:, :], pad_rows//2, axis=0)
                x_padded = tf.concat([padding_front, x_padded, padding_back], axis=0)
            elif PADDING == 5: #copy last frame
                padding_back = tf.repeat(x_padded[-1:, :], pad_rows, axis=0)
                x_padded = tf.concat([x_padded, padding_back], axis=0)
            else:
                if PADDING ==1: #padding at start and end
                    if pad_rows %2 == 0: #if pad_rows is even
                        paddings = [[pad_rows//2, pad_rows//2], [0, 0], [0, 0]]
                    else: #if pad_rows is odd
                        paddings = [[pad_rows//2+1, pad_rows//2], [0, 0], [0, 0]]
                elif PADDING ==2: #padding only at the end of sequence
                    paddings = [[0, pad_rows], [0, 0], [0, 0]]
                x_padded = tf.pad(x_padded, paddings, mode='CONSTANT', constant_values=CONSTANT_VALUE)

        x = x_padded
        current_rows = tf.shape(x)[0]

        #interpolate single missing values
        x = pd.DataFrame(np.array(x).flatten()).interpolate(method='linear', limit=2, limit_direction='both')
        #fill missing values with zeros
        x = tf.where(tf.math.is_nan(x), tf.zeros_like(x), x)
        
        #reshape data to 2D or 3D array
        if FLATTEN:
            x = tf.reshape(x, (1, current_rows*INPUT_SHAPE[1]))
        else:
            x = tf.reshape(x, (1, current_rows, INPUT_SHAPE[1]))

        return x

#define converter using generated layer
feature_converter = FeatureGen()

## Real Time Prediction / Detection

Press "Q" to interrupt the camera feed. 

In [690]:
#initial setup
sequence = [] # to collect all 22 frames for prediction
predictions = []
threshold = 0.5 # confidence metrics (only render prediction results, if confidence is above threshold)
sentence = []
sign_predicted = '...'
stage = 0 # stage of the game level

cap = cv2.VideoCapture(0) # grabbing webcam
MODE = -1 # initial mode for menu
L_RENDER = 0 # initial state without render
PREDICT = 0

# set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: 
    while cap.isOpened(): # loop through all frames 
        key = cv2.waitKey(10) & 0xFF
        # read feed
        ret, frame = cap.read()
        # lower quality for zoom transmission
        #frame = cv2.resize(frame, (0,0), fx = 0.75, fy = 0.75)
        # make detections 
        image, results = mediapipe_detection(frame, holistic)

        #draw_landmarks(image, results)
        if L_RENDER == 1: 
            draw_styled_landmarks(image, results)

        # 2. Prediction logic
        keypoints = extract_keypoints(results) # extract keypoints x, y, z for face, left_hand, pose, right_hand from mediapipe holistic predictions, keypoints.shape e.g. (543, 3)

        # Start menu
        if MODE == -1: 

            # transparent box
            overlay = image.copy()
            cv2.rectangle(image, (0, 0), (1280, 100), color_coral, -1)
            cv2.rectangle(image, (0, 100), (1280, 620), color_turquoise, -1)
            cv2.rectangle(image, (0, 620), (1280, 720), color_coral, -1)
            alpha = 0.5  # Transparency factor

            # Following line overlays transparent rectangle over the image
            image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

            text_indent = 300
            cv2.putText(image, 'SignMeUp!', (text_indent+50, 175+50), cv2.FONT_HERSHEY_SIMPLEX, 4, color_bg, 6, cv2.LINE_AA)
            cv2.putText(image, '(Demo)', (text_indent+250, 230+50), cv2.FONT_ITALIC, 1, color_bg, 1, cv2.LINE_AA)
            cv2.putText(image, 'Press "0" for basic mode', (text_indent+50, 300+35), cv2.FONT_ITALIC, 1, color_coral, 2, cv2.LINE_AA)
            cv2.putText(image, 'Press "1" for advanced mode', (text_indent+50, 340+35), cv2.FONT_ITALIC, 1, color_coral, 2, cv2.LINE_AA)
            cv2.putText(image, 'Press "g" for playing the tutorial game', (text_indent+50, 380+35), cv2.FONT_ITALIC, 1, color_coral, 2, cv2.LINE_AA)
            cv2.putText(image, 'Press "r" to reset', (text_indent+50, 420+35), cv2.FONT_ITALIC, 1, color_coral, 2, cv2.LINE_AA)
            cv2.putText(image, 'Press "m" to return to menu', (text_indent+50, 460+35), cv2.FONT_ITALIC, 1, color_coral, 2, cv2.LINE_AA)
            cv2.putText(image, 'Press "l" to turn on/off landmarks', (text_indent+50, 500+35), cv2.FONT_ITALIC, 1, color_coral, 2, cv2.LINE_AA)
            cv2.putText(image, '(C) 2023 SIGNMEUP', (text_indent+175, 550+50), cv2.FONT_HERSHEY_PLAIN, 2, color_bg, 2, cv2.LINE_AA)


            cv2.imshow("SignMeUp Real-Time Demo", image)
            
            #change modes
            if key == ord('1'): 
                MODE = 1
            elif key == ord('0'): 
                MODE = 0
            elif key == ord('g'): 
                MODE = 2
            elif key == ord('q'): 
                break

        # Live mode
        elif MODE == 0 or MODE == 1 or MODE == 2:
            if MODE == 1:
                image = prob_viz(pred, image, SELECTED_SIGNS)
            if (results.left_hand_landmarks or results.right_hand_landmarks) and PREDICT: #check if hand landmarks are present: 
                sequence.append(keypoints) # keep appending keypoints (frames) to a sequence, np.array(sequence).shape e.g. (22, 543, 3)
                # pre-processing
                model_input = feature_converter(np.array(sequence))

                # prediction
                pred = model.predict(model_input)[0] # model.fit() expects something in shape (num_sequences, 30, 1662), e.g. (1, 30, 1662) for a single sequence
                selected_labels = [LABEL_MAP[x] for x in SELECTED_SIGNS]
                pred = pred[selected_labels]
                sign_predicted = '...'
            
            else: #no hands visible
                if len(sequence) > 2 : #run predictions if sequence was created

                    # pre-processing
                    model_input = feature_converter(np.array(sequence))

                    # prediction
                    pred = model.predict(model_input)[0] # model.fit() expects something in shape (num_sequences, 30, 1662), e.g. (1, 30, 1662) for a single sequence
                    selected_labels = [LABEL_MAP[x] for x in SELECTED_SIGNS]
                    pred = pred[selected_labels]

                    # if the confidence of the most confident prediction is above threshold
                    if pred[np.argmax(pred)] > threshold: 
                        if MODE == 2: #game mode
                            level_labels = [LABEL_MAP.get(x) for x in LEVEL[stage]] 
                            # filter predictions for only signs used in level
                            pred_level = []
                            for i in range(len(pred)): 
                                if i in level_labels: 
                                    pred_level.append(pred[i])
                                else: 
                                    pred_level.append(0)
                            # find sign with max probability
                            sign_predicted = SELECTED_SIGNS[np.argmax(pred_level)]
                            sentence.append(sign_predicted)
                        else:
                            sentence = []
                            sign_predicted = SELECTED_SIGNS[np.argmax(pred)]

                    else: #if threshold is not reached
                        sign_predicted = '...'
                    sequence = [] #resetting sequence for next iteration
                else:
                    pred = 20 *[0]
            
            # rendering of predicted sign/word at the top of the feed inside a textbox (whole width: 1280)
            overlay = image.copy()
            cv2.rectangle(image, (0, 0), (230, 120), color_turquoise, -1)
            alpha = 0.5  # Transparency factor
            # Following line overlays transparent rectangle over the image
            image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)

            if PREDICT: 
                cv2.putText(image, 'You signed:', (10, 40), cv2.FONT_ITALIC, 0.75, color_bg, 1, cv2.LINE_AA)
                cv2.putText(image, '"'+sign_predicted+'"', (10, 90), cv2.FONT_ITALIC, 1, color_bg, 2, cv2.LINE_AA)
            else: 
                cv2.putText(image, '(press p to start)', (10, 40), cv2.FONT_ITALIC, 0.75, color_bg, 1, cv2.LINE_AA)
            
            # formatting of level boxes and texts
            larea_start = 240
            larea_end = 1280
            lbox_counts = 5
            lbox_spacing = 5
            lbox_step = (larea_end - larea_start)/lbox_counts
            lbox_width = lbox_step - lbox_spacing
            lbox_x_start = [larea_start + n*lbox_step  for n in range(lbox_counts)]
            lbox_x_end = [lbox_x_start[n] + lbox_width for n in range(lbox_counts)]

            if MODE == 2: # Game mode
                cv2.putText(image, f'Level: {stage}', (1125, 700), cv2.FONT_ITALIC, 1, color_coral, 2, cv2.LINE_AA)
                cv2.imshow("SignMeUp Real-Time Demo", image)
                for i in range(len(LEVEL[0])): 
                    # box showing the sign to perform in this "level" 
                    overlay = image.copy() # for transparency effect
                    alpha = 0.5  # Transparency factor

                    if LEVEL[stage][i] in sentence: #if sign is in sentence, it will be colored
                        cv2.rectangle(overlay, (int(lbox_x_start[i]), 0), (int(lbox_x_end[i]), 60), color_turquoise, -1)
                        image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0) # overlays transparent rectangle over the image
                        cv2.putText(image, LEVEL[stage][i], (int(lbox_x_start[i])+10, 40), cv2.FONT_ITALIC, 1, color_bg, 1, cv2.LINE_AA)
                    else: 
                        cv2.rectangle(overlay, (int(lbox_x_start[i]), 0), (int(lbox_x_end[i]), 60), color_coral, -1)
                        image = cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0) # overlays transparent rectangle over the image
                        cv2.putText(image, LEVEL[stage][i], (int(lbox_x_start[i])+10, 40), cv2.FONT_ITALIC, 1, color_bg, 2, cv2.LINE_AA)
                    
                    
                if all(item in sentence for item in LEVEL[stage]): #checking if level is cleared
                    if stage == len(LEVEL)-1: #end of tutorial
                        cv2.putText(image, 'CONGRATULATIONS!!!', (10, 400), cv2.FONT_ITALIC, 4, color_coral, 10, cv2.LINE_AA)
                        cv2.putText(image, 'You finished the tutorial!', (250, 500), cv2.FONT_ITALIC, 2, color_coral, 5, cv2.LINE_AA)
                        cv2.putText(image, 'Press "m" to return to menu!', (350, 600), cv2.FONT_ITALIC, 1, color_turquoise, 2, cv2.LINE_AA)
                        cv2.imshow("SignMeUp Real-Time Demo", image)
                    else: #iterating to next level
                        stage += 1
                        sentence = []
                        cv2.putText(image, 'Level cleared!!!', (175, 400), cv2.FONT_ITALIC, 4, color_coral, 10, cv2.LINE_AA)
                        cv2.putText(image, 'Press any key to continue to the next level.', (300, 500), cv2.FONT_ITALIC, 1, color_turquoise, 2, cv2.LINE_AA)
                        cv2.imshow("SignMeUp Real-Time Demo", image)
                        cv2.waitKey(0)
                            
        # show current frame to screen
        cv2.imshow("SignMeUp Real-Time Demo", image)

        #change modes
        if key == ord('1'): 
            MODE = 1
        elif key == ord('0'): 
            MODE = 0
        elif key == ord('l'): 
            L_RENDER = not L_RENDER 
        elif key == ord('g'): 
            MODE = 2
        elif key == ord('r'): 
            stage = 0
            sentence = []
            sign_predicted = "..."
        elif key == ord('m'): 
            MODE = -1
        elif key == ord('p'): 
            PREDICT = not PREDICT
        elif key == ord('q'): 
            break

        
# release camera and close feed window 
cap.release()
cv2.destroyAllWindows() 
cv2.waitKey(1) # some workaround to fix the bug, that window doesn't close
            

-1